<a href="https://colab.research.google.com/github/premanshsharma/Face-Recognition-/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
from sklearn.preprocessing import OneHotEncoder 
classes = []
for r, d, f in os.walk(r'/content/gdrive/MyDrive/Face_Recognition /Original_Image_Unzip/Original Images'):
    classes.append([d, f, r])
# print(classes)
class_name = classes[0][0]
class_name = [[class_name[i], i] for i in range(len(class_name))]
enc = OneHotEncoder()
enc.fit(class_name)
OneHotEncoder(handle_unknown='ignore')
data, result = [], []
map = {}
for i in class_name:
    map[i[0]] = []
for r, d, f in os.walk(r'/content/gdrive/MyDrive/Face_Recognition /Original_Image_Unzip/Original Images'):
    for file in f:
        if '.jpg' in file:
            k = ''
            for i in file:
                if i == '_':
                    break
                k += i
            map[k].append(os.path.join(r, file))


In [ ]:
from PIL import Image
import numpy as np
data, result = [], []
i = 0
for photos in map:
    # print(photos, map[photos])
    for path in map[photos]:
        # print(path)
        img = Image.open(path)
        img = img.resize((224,224))
        img = np.array(img)
        if(img.shape == (224,224,3)):
            data.append(np.array(img))
            result.append(i)
    i += 1

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 206 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 724 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 170 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 178 bytes but o

In [ ]:
for i in range(result[i]):
    # result[i] = encoder.transform([[result[i]]]]).toarray()
    result[i] = encoder.transform([[result[i]]]).toarray()
print(len(data), len(result))

2562 2562


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, result, test_size=0.2, shuffle = True, random_state = 8)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state= 8)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
print(len(x_train), len(x_test), len(x_val))

1536 513 513


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers.merge import concatenate
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer

def GoogLeNet():
  # input layer 
  import tensorflow as tf
  #   input_layer = Input(shape = (224, 224, 3))
  input_layer = tf.keras.Input(shape=(224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(5, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(1000, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(1000, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [ ]:
def vgg16():
    model1 = Sequential()
    model1.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model1.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model1.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model1.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model1.add(Flatten())
    model1.add(Dense(units=4096,activation="relu"))
    model1.add(Dense(units=4096,activation="relu"))
    model1.add(Dense(units=len(map), activation="softmax"))

    from tensorflow.keras.optimizers import Adam 
    opt = Adam(lr=0.001)
    model1.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model1

In [ ]:
model = vgg16()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(x_train, y_train, epochs = 100, batch_size = 40, validation_data = (x_val, y_val))

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


39/39 [==============================] - 43s 850ms/step - loss: 94.4157 - accuracy: 0.0410 - val_loss: 3.4125 - val_accuracy: 0.0429
Epoch 2/100
39/39 [==============================] - 25s 630ms/step - loss: 3.3998 - accuracy: 0.0508 - val_loss: 3.4025 - val_accuracy: 0.0546
Epoch 3/100
39/39 [==============================] - 24s 627ms/step - loss: 3.4032 - accuracy: 0.0469 - val_loss: 3.4092 - val_accuracy: 0.0292
Epoch 4/100
39/39 [==============================] - 25s 632ms/step - loss: 3.4027 - accuracy: 0.0423 - val_loss: 3.4027 - val_accuracy: 0.0429
Epoch 5/100
39/39 [==============================] - 25s 630ms/step - loss: 3.3970 - accuracy: 0.0391 - val_loss: 3.4062 - val_accuracy: 0.0429
Epoch 6/100
39/39 [==============================] - 24s 629ms/step - loss: 3.3973 - accuracy: 0.0495 - val_loss: 3.4043 - val_accuracy: 0.0429
Epoch 7/100
39/39 [==============================] - 24s 629ms/step - loss: 3.3970 - accuracy: 0.0495 - val_loss: 3.4054 - val_accuracy: 0.0429
Epo

KeyboardInterrupt: ignored

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
9/9 [==============================] - 3s 141ms/step - loss: nan - dense_45_loss: nan - dense_42_loss: nan - dense_44_loss: nan - dense_45_accuracy: 0.0058 - dense_42_accuracy: 0.0058 - dense_44_accuracy: 0.0058
test loss, test acc: [nan, nan, nan, nan, 0.005847953259944916, 0.005847953259944916, 0.005847953259944916]
